In [1]:
import joblib
m= joblib.load("xgboost_model.pkl")

In [2]:
import gradio as gr
import numpy as np


season_map = {
    "IPL-2008": 0, "IPL-2009": 1, "IPL-2010": 2, "IPL-2011": 3,
    "IPL-2012": 4, "IPL-2013": 5, "IPL-2014": 6, "IPL-2015": 7,
    "IPL-2016": 8, "IPL-2017": 9, "IPL-2018": 10, "IPL-2019": 11
}

city_map = {
    "Abu Dhabi": 0, "Ahmedabad": 1, "Bangalore": 2, "Bengaluru": 3,
    "Bloemfontein": 4, "Cape Town": 5, "Centurion": 6, "Chandigarh": 7,
    "Chennai": 8, "Cuttack": 9, "Delhi": 10, "Dharamsala": 11, "Durban": 12,
    "East London": 13, "Hyderabad": 14, "Indore": 15, "Jaipur": 16,
    "Johannesburg": 17, "Kanpur": 18, "Kimberley": 19, "Kochi": 20,
    "Kolkata": 21, "Mohali": 22, "Mumbai": 23, "Nagpur": 24,
    "Port Elizabeth": 25, "Pune": 26, "Raipur": 27, "Rajkot": 28,
    "Ranchi": 29, "Sharjah": 30, "Visakhapatnam": 31
}

toss_winner_map = {
    "Chennai Super Kings": 0, "Deccan Chargers": 1, "Delhi Capitals": 2,
    "Delhi Daredevils": 3, "Gujarat Lions": 4, "Kings XI Punjab": 5,
    "Kochi Tuskers Kerala": 6, "Kolkata Knight Riders": 7,
    "Mumbai Indians": 8, "Pune Warriors": 9, "Rajasthan Royals": 10,
    "Rising Pune Supergiant": 11, "Rising Pune Supergiants": 12,
    "Royal Challengers Bangalore": 13, "Sunrisers Hyderabad": 14
}

result_map = {"no result": 0, "normal": 1, "tie": 2}

# Example win_by_runs mapping (extend as you provided)
win_by_runs_map = {
    "0": 0, "1": 1, "10": 2, "102": 3, "105": 4, "11": 5, "111": 6,
    "118": 7, "12": 8, "13": 9, "130": 10, "138": 11, "14": 12, "140": 13,
    "144": 14, "146": 15, "15": 16, "16": 17, "17": 18, "18": 19,
    "19": 20, "2": 21, "20": 22, "21": 23, "22": 24, "23": 25, "24": 26,
    "25": 27, "26": 28, "27": 29, "28": 30, "29": 31, "3": 32, "30": 33,
    "31": 34, "32": 35, "33": 36, "34": 37, "35": 38, "36": 39,
    "37": 40, "38": 41, "39": 42, "4": 43, "40": 44, "41": 45, "42": 46,
    "43": 47, "44": 48, "45": 49, "46": 50, "47": 51, "48": 52,
    "5": 53, "50": 54, "51": 55, "53": 56, "55": 57, "57": 58,
    "58": 59, "59": 60, "6": 61, "60": 62, "61": 63, "62": 64, "63": 65,
    "64": 66, "65": 67, "66": 68, "67": 69, "7": 70, "71": 71, "72": 72,
    "74": 73, "75": 74, "76": 75, "77": 76, "78": 77, "8": 78,
    "80": 79, "82": 80, "85": 81, "86": 82, "87": 83, "9": 84,
    "92": 85, "93": 86, "97": 87, "98": 88
}

venue_map = {
    "ACA-VDCA Stadium": 0, "Barabati Stadium": 1, "Brabourne Stadium": 2,
    "Buffalo Park": 3, "De Beers Diamond Oval": 4, "Dr DY Patil Sports Academy": 5,
    "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium": 6,
    "Dubai International Cricket Stadium": 7, "Eden Gardens": 8,
    "Feroz Shah Kotla": 9, "Feroz Shah Kotla Ground": 10, "Green Park": 11,
    "Himachal Pradesh Cricket Association Stadium": 12, "Holkar Cricket Stadium": 13,
    "IS Bindra Stadium": 14, "JSCA International Stadium Complex": 15,
    "Kingsmead": 16, "M Chinnaswamy Stadium": 17, "M. A. Chidambaram Stadium": 18,
    "M. Chinnaswamy Stadium": 19, "MA Chidambaram Stadium, Chepauk": 20,
    "Maharashtra Cricket Association Stadium": 21, "Nehru Stadium": 22,
    "New Wanderers Stadium": 23, "Newlands": 24, "OUTsurance Oval": 25,
    "Punjab Cricket Association IS Bindra Stadium, Mohali": 26,
    "Punjab Cricket Association Stadium, Mohali": 27,
    "Rajiv Gandhi International Stadium, Uppal": 28,
    "Rajiv Gandhi Intl. Cricket Stadium": 29, "Sardar Patel Stadium, Motera": 30,
    "Saurashtra Cricket Association Stadium": 31, "Sawai Mansingh Stadium": 32,
    "Shaheed Veer Narayan Singh International Stadium": 33,
    "Sharjah Cricket Stadium": 34, "Sheikh Zayed Stadium": 35,
    "St George's Park": 36, "Subrata Roy Sahara Stadium": 37,
    "SuperSport Park": 38, "Vidarbha Cricket Association Stadium, Jamtha": 39,
    "Wankhede Stadium": 40
}

team_state_map = {
    "Bangalore": 0, "Chennai": 1, "Delhi": 2, "Gujarat": 3,
    "Hyderabad": 4, "Kerala": 5, "Kolkata": 6, "Mumbai": 7,
    "Pune": 8, "Punjab": 9, "Rajasthan": 10
}

def predict_winner(season, city, date, toss_winner, toss_decision, result,
                  dl_applied, win_by_runs, win_by_wickets, venue,
                  team1_state, team2_state):

    features = np.array([[
        season_map[season],
        city_map[city],
        date,
        toss_winner_map[toss_winner],
        toss_decision,
        result_map[result],
        dl_applied,
        win_by_runs_map[win_by_runs],
        win_by_wickets,
        venue_map[venue],
        team_state_map[team1_state],
        team_state_map[team2_state],
       
    ]])

    prediction = m.predict(features)[0]

    predicted_team = [team for team, code in team_state_map.items() if code == prediction][0]

    return f"Predicted Winning Team State: {predicted_team}"


demo = gr.Interface(
    fn=predict_winner,
    inputs=[
        gr.Dropdown(list(season_map.keys()), label="Season"),
        gr.Dropdown(list(city_map.keys()), label="City"),
        gr.Number(label="Date"),
        gr.Dropdown(list(toss_winner_map.keys()), label="Toss Winner"),
        gr.Radio([0,1], label="Toss Decision (0=bat,1=field)"),
        gr.Dropdown(list(result_map.keys()), label="Result"),
        gr.Radio([0,1], label="DL Applied (0=No,1=Yes)"),
        gr.Dropdown(list(win_by_runs_map.keys()), label="Win by Runs"),
        gr.Dropdown([0,1,2,3,4,5,6,7,8,9,10,11],label="Win By Wickets"),
        gr.Dropdown(list(venue_map.keys()), label="Venue"),
        gr.Dropdown(list(team_state_map.keys()), label="Team 1 State"),
        gr.Dropdown(list(team_state_map.keys()), label="Team 2 State"),
        
    ],
    outputs="text",
    title="IPL Match Winner Predictor"
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
import gradio as gr
import numpy as np

# Dictionary to map encoded output back to team names
team_mapping = {
    0: "Bangalore",
    1: "Chennai",
    2: "Delhi",
    3: "Gujarat",
    4: "Hyderabad",
    5: "Kerala",
    6: "Kolkata",
    7: "Mumbai",
    8: "Pune",
    9: "Punjab",
    10: "Rajasthan",
    11: "Unknown/NaN"
}

# Prediction function
def predict_match(season, city, date, toss_winner, toss_decision, result,
                  dl_applied, win_by_runs, win_by_wickets, venue,
                  team1_state, team2_state):
    
    # Arrange input features in the same order as training
    features = np.array([[season, city, date, toss_winner, toss_decision,
                          result, dl_applied, win_by_runs, win_by_wickets,
                          venue, team1_state, team2_state]])
    
    # Get numeric prediction
    pred_num = m.predict(features)[0]
    
    # Map back to team name
    return f"Predicted Winner: {team_mapping.get(pred_num, 'Unknown')}"

# Gradio interface
interface = gr.Interface(
    fn=predict_match,
    inputs=[
        gr.Number(label="Season"),
        gr.Number(label="City"),
        gr.Number(label="Date"),
        gr.Number(label="Toss Winner"),
        gr.Radio([0,1], label="Toss Decision (0=bat,1=field)"),
        gr.Number(label="Result"),
        gr.Radio([0,1], label="DL Applied (0=No,1=Yes)"),
        gr.Number(label="Win By Runs"),
        gr.Number(label="Win By Wickets"),
        gr.Number(label="Venue"),
        gr.Number(label="Team 1 State"),
        gr.Number(label="Team 2 State"),
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="IPL Match Winner Prediction",
    description="Enter match details to predict the winning team."
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
